In [1]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Project")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/14 22:32:20 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.31.190.2 instead (on interface eth0)
22/09/14 22:32:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/14 22:32:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import geopandas as gpd

# load postcode shapefile
# TODO: change this to postcode
sf = gpd.read_file("../../../data/tables/external_datasets/postcode_districts/POA_2021_AUST_GDA94.shp")

In [3]:
sf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   POA_CODE21  2644 non-null   object  
 1   POA_NAME21  2644 non-null   object  
 2   AUS_CODE21  2644 non-null   object  
 3   AUS_NAME21  2644 non-null   object  
 4   AREASQKM21  2644 non-null   float64 
 5   LOCI_URI21  2644 non-null   object  
 6   SHAPE_Leng  2644 non-null   float64 
 7   SHAPE_Area  2644 non-null   float64 
 8   geometry    2641 non-null   geometry
dtypes: float64(3), geometry(1), object(5)
memory usage: 186.0+ KB


In [4]:
# cast shapefile postcode col to int to match sdf
sf['POA_CODE21'] = sf['POA_CODE21'].astype('int')

ValueError: invalid literal for int() with base 10: 'ZZZZ'

In [ ]:
# thats freakin weird -> wtf is ZZZZ ahahah thats getting removed
sf['POA_CODE21'].tail()

In [ ]:
import pandas as pd
# remove ZZZZ/non numeric entry
sf = sf[pd.to_numeric(sf['POA_CODE21'], errors='coerce').notnull()]
sf.tail()

In [ ]:
# cast to int again
sf['POA_CODE21'] = sf['POA_CODE21'].astype('int')
sf.head()

In [ ]:
# import sdf
sdf = spark.read.parquet("../../../data/curated/process_data.parquet/")

In [ ]:
# cast postcode to int to match the format of gdf
sdf = sdf.withColumn('postcode', sdf['postcode'].cast('int'))

In [ ]:
#transactions = spark.read.parquet('../../../data/tables/transactions_20210228_20210827_snapshot')

In [ ]:
sdf.printSchema()

In [ ]:
sdf.orderBy('postcode')

In [ ]:
#statepd = sdf.groupBy("postcode").mean("dollar_value")
postcode_dv = sdf.groupBy("postcode").mean("dollar_value").toPandas()

# .toPandas()

In [ ]:
postcode_dv.info()
postcode_dv.describe()

In [ ]:
postcode_dv.sort_values('postcode')

In [ ]:
sf

In [ ]:
#sf['postcode'] = sf['postcode'].astype('int32')

In [ ]:
postcode_join_sf = postcode_dv \
    .merge(sf[['POA_CODE21', 'geometry']], left_on='postcode', right_on='POA_CODE21') \
    .drop('POA_CODE21', axis=1)

In [ ]:
postcode_join_sf.sort_values('postcode')

In [ ]:
sf.plot()

In [ ]:
gdf = gpd.GeoDataFrame(postcode_join_sf)

In [ ]:
gdf.geometry.name

Finally draw tha plot ting

In [30]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
# plot the avg dollar values by postcode
fig, ax2 = plt.subplots(1, figsize=(32,24), dpi=600)

# align legend with plot
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.1)

# plot
gdf.plot(column=('avg(dollar_value)'),
         legend=True,
         #legend_kwds={'label' : 'Average Dollar Value by Postcode',
         #               'orientation' : 'horizontal'},
         ax=ax2,
         cax=cax)

plt.show()

#gdf.plot(column=('avg(dollar_value)'),legend=True, figsize=(32,24))

NameError: name 'make_axes_locatable' is not defined